##### *Reminder - use daily log to track progress

In [ ]:
!pip install librosa

In [440]:
import pandas as pd
import numpy as np
import os, pathlib
from sklearn.model_selection import train_test_split
import librosa
import torch
import IPython.display as ipd
from random import randint

In [ ]:
!sudo apt-get install wget unzip

In [ ]:
# Download training data
!mkdir -p data && cd data && wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TrainingData.zip && unzip IRMAS-TrainingData.zip 

In [ ]:
#Download testing data (Around 2GB each)
wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part1.zip && IRMAS-TestingData-Part1.zip
wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part2.zip && IRMAS-TestingData-Part2.zip
wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part3.zip && IRMAS-TestingData-Part3.zip    

IRMAS dataset (training)
 
Audio files: 6705 audio files in 16 bit stereo wav format sampled at 44.1kHz. They are excerpts of 3 seconds from more than 2000 distinct recordings. 

Annotations: The annotation of the predominant instrument of each excerpt is both in the name of the containing folder, and in the file name: cello (cel), clarinet (cla), flute (flu), acoustic guitar (gac), electric guitar (gel), organ (org), piano (pia), saxophone (sax), trumpet (tru), violin (vio), and human singing voice (voi). The number of files per instrument are: cel(388), cla(505), flu(451), gac(637), gel(760), org(682), pia(721), sax(626), tru(577), vio(580), voi(778). 

Additionally, some of the files have annotations in the filename regarding the presence ([dru]) or non presence([nod]) of drums, and the musical genre: country-folk ([cou_fol]), classical ([cla]), pop-rock ([pop-roc]), latin-soul ([lat-sou]).

In [89]:
IRMAS_TRAINING = 'data/IRMAS-TrainingData'
base_path = pathlib.Path(IRMAS_TRAINING)
classes, paths = [], []

for p in base_path.glob('*/*'):
    relative_path = p.relative_to(base_path)
    classes.append(str(relative_path.parent))
    paths.append(p)
    
#classes = set(classes)
df = pd.DataFrame({"tags": classes, "wav_path": paths}).sample(frac=1).reset_index()
#df_training.piano = df.tags.map({'pia': True, '': 0})
#df.tags = df.tags.map({'pia': True, })

df.head(5)

,index,tags,wav_path
0,800,voi,data/IRMAS-TrainingData/voi/058__[voi][nod][co...
1,2944,org,data/IRMAS-TrainingData/org/[org][pop_roc]1128...
2,3918,gac,data/IRMAS-TrainingData/gac/[gac][jaz_blu]0607...
3,1561,tru,data/IRMAS-TrainingData/tru/[tru][pop_roc]2046...
4,6287,gel,data/IRMAS-TrainingData/gel/[gel][pop_roc]0902...


In [84]:
df.shape

(6705, 3)

In [91]:
classes = set(classes)

In [92]:
classes

{'cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi'}

In [93]:
classes = list(classes)

In [95]:
type(classes)

list

In [57]:
#inst_dict = {1:'cel', 2:'cla', 3:'flu', 4:'gac', 5:'gel',6:'org',7:'pia',8:'sax',9:'tru',10:'vio',11:'voi'}

In [111]:
class_dict = { i: classes[i] for i in range(0, len(classes))}

In [113]:
class_dict

{0: 'vio',
 1: 'gac',
 2: 'cel',
 3: 'pia',
 4: 'sax',
 5: 'gel',
 6: 'flu',
 7: 'voi',
 8: 'cla',
 9: 'org',
 10: 'tru'}

In [114]:
class_dict[9]

'org'

In [107]:
df_train, df_test = train_test_split(df, test_size=0.3)

In [108]:
df_train.shape

(4693, 3)

In [109]:
df_test.shape

(2012, 3)

In [110]:
# Test if files are loading
successful, corrupted = [], []
len_x, sample_rates = [], []
for p in df.wav_path:
    try:
        x, sr = librosa.load(p, sr=None)
        successful.append(p)
        len_x.append(len(x))
        sample_rates.append(sr)
    except:
        corrupted.append(p)
#       print(p)
###df = df[~df.wav_path.isin(corrupted)]
len(successful), len(corrupted)

(6705, 0)

In [121]:
set(len_x), set(sample_rates), 132299/44100

({132299}, {44100}, 2.9999773242630385)

In [153]:
class InstrumentClassificationDataset():
    def __init__(self, df):
        super().__init__()
        self.df = df.copy()
#        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        sound = self.load_from_disk(index)
        label = self.load_label(index)
#        Xi = self.image_transform(Xi)
        return sound, label

    def load_to_librosa(self, path):
        image = librosa.load(path)
        return image

    def load_from_disk(self, index):
        wav_path = df.iloc[index].wav_path 
        return self.load_to_librosa(wav_path)

    def load_label(self, index):
        label = df.iloc[index].tags
        return label

In [47]:
class BaseSampler():
    def __init__(self, df, list_of_instruments, n_samples):
        self.df = df.copy()
        self.n_samples = n_samples
        self.instruments = list_of_instruments
        #df = df[self.instruments]
        df = df[df['tags'].isin(inst)]
        
    def __iter__(self):
        return iter(self._get_sample())
        
    def __len__(self):
        return self.n_samples
    
    def _get_samples(self):
        return np.random.choice(len(self.df), self.n_samples, replace=False)

In [168]:
# define the instruments we are using
list_of_instruments = ['sax', 'pia']

In [169]:
bs = BaseSampler(df, list_of_instruments, 30)

In [192]:
bs._get_samples()

array([ 941, 3133, 2640,  871, 2903, 4623, 1271, 5621, 2398, 6684, 6689,
       5033,  442,  990, 6458, 3804, 6054,  816, 1363, 6364, 3132, 4852,
       2393, 5993, 5662, 2018, 3580, 4230,  589, 6549])

In [178]:
batch_index = bs._get_samples()

In [171]:
ds_train = InstrumentClassificationDataset(df_train)

((array([ 0.14183977,  0.11393391,  0.09872768, ..., -0.05704049,
         -0.10404866,  0.        ], dtype=float32),
  22050),
 'gel')

In [401]:
def sound_generator(df, inst, batch_size = 30):
    df = df[df['tags'].isin(inst)].copy()
    while True:
        batch_x, labels = [], []
        batch_index = np.random.choice(len(df), batch_size, replace=False)
        for idx in batch_index:
            wav_path = df.iloc[idx].wav_path 
            sound_frame, sr = librosa.load(wav_path, sr=None)
            batch_x.append(sound_frame)
            labels.append(df.iloc[idx].tags)
#        batch_x = np.array( batch_x )
#        labels = np.array (labels)
        yield( batch_x, labels )

In [402]:
sg  = sound_generator(df_train, list_of_instruments, 30)

In [451]:
sounds, labels = next(sg)

In [452]:
sounds = np.array(sounds)

In [453]:
sounds.shape

(30, 132299)

In [454]:
sounds[1], labels[1]

(array([-0.00326538,  0.00131226, -0.00419617, ...,  0.04437256,
         0.03317261,  0.07290649], dtype=float32),
 'sax')

In [445]:
len(sounds[1]), min(sounds[1]), max(sounds[1])

(132299, -0.5867615, 0.56440735)

In [446]:
# Check if the sounds and labels match 
random = randint(0,29)
print(labels[random])
ipd.Audio(sounds[random], rate=44100)

pia
